In [25]:
import pandas as pd
import numpy as np
import SETTINGS as s
#затягиваем настройки
cal = pd.read_excel('meta_settings/calendar.xlsx')
grades_trans = pd.read_excel('meta_settings/grades_translation.xlsx')
#затягиваем данные
cons = pd.read_excel('input/consumption.xlsx')
cur_inv = pd.read_excel('input/inventory.xlsx')
orders = pd.read_excel('input/orders.xlsx')
#consN.to_excel('check.xlsx')

In [26]:
#prepare, тут лучше работать через календарь - исправить
calW=cal.ix[:,1:].drop_duplicates()
cons = cons.merge(calW,how='left',on=['year','week'])
clear = pd.DataFrame(cons.week_acc.drop_duplicates().reset_index(drop=True))
clear['week_clear']=clear.index+1
cons = cons.merge(clear,how='left',on=['week_acc'])

In [27]:
#pivot consumption
consPivC = pd.pivot_table(cons, columns = 'week_clear', index=['grade','gauge','width'], values='quantity', aggfunc=np.sum,fill_value=0)
consPiv = pd.pivot_table(cons, columns = 'week_acc', index=['grade','gauge','width'], values='quantity', aggfunc=np.sum,fill_value=0)

In [28]:
#методы прогноза, возвращают табличку с прогнозом
def forecast_naive1week(consPiv, lt):
    Forecast = pd.DataFrame(index = consPiv.index, data = consPiv.values, columns = consPiv.columns+lt)
    return Forecast    

In [29]:
#делаю прогноз
forecast = forecast_naive1week(consPivC, s.lt)

In [30]:
#выбираю отрезок, на котором буду считать страховой запас
st = max(forecast.columns.min(),consPivC.columns.min())
end = min(forecast.columns.max(),consPivC.columns.max())

In [31]:
#табличка с отклонениями прогноза
deviation = (forecast.ix[:,st:end] - consPivC.ix[:,st:end])
deviation[deviation<0]=0

In [32]:
#делаем табличку для занесения safety stock
safety_table = pd.Series(index = consPivC.index)

In [33]:
#считаем страховой запас lt
DEVs = deviation.iterrows()
for x in range(len(deviation)):
    r = next(DEVs)[1].rolling(window=s.lt)
    ss = r.sum().max()
    safety_table[x] = ss

In [34]:
#считаем, что будет потребляться в теч лид тайма
cons_lt = forecast.ix[:,s.start_week:(s.start_week+s.lt)]
cons_lt = cons_lt.stack().groupby(level=[0,1,2]).sum()

In [35]:
#затягиваем и подготавливаем заказы
orders = orders.merge(cal,how='left',on=['date'])
orders = orders.merge(grades_trans,how='left',on=['gradeLIP'])

In [36]:
#выбираем последние заказы и получаем товары в пути 
in_transit = orders.loc[orders['week_acc'].isin(range(s.start_week-s.lt,s.start_week))]
in_transit = in_transit.pivot_table(index = ['gradeUS','gauge','width'],values = 'qMt',aggfunc=np.sum)
in_transit.index.rename(['grade','gauge','width'], inplace = True)


In [37]:
#текущий запас
import matplotlib
consPivC[1].plot(kind='bar')

In [38]:
#считаем запас на момент поставки текущего заказа

In [39]:
#считаем потребность, исходя из прогноза на период [lead time:lead time+interval]

In [40]:
#формируем судовую партию

In [41]:
#формируем заказ